I have a prototype of they system for corrupting sentences, for different semantic meaning:

My process is:

1. Tokenize the sentence (Currently using the NLTK regex tokenizer, it seem sufficient)
2. Parts of Speech tag (Currently using the Stanford POS Tagger (via NLTK))
3. For each word that is not blacklisted (I have blacklisted "had", "were", "have", "was", and "be", as they are unusual verbs with strange antoymns, further more the are rather syntatic)
        1. Use WordNet to find antonyms of the same POS tag  (So "race" (Noun, as in a competition) has no antonyms, but "race" (Verb, to race the train) has "linger" as an antonym.
        2. Unstem: WordNet stemming/lemmaisation (of the antonym) removes Tense, Plurality, comparativeness, and superlativeness, so I make use of the POS tag of the original to work those out, then restore them using the Pattern library's tools for this (http://jmlr.csail.mit.edu/papers/volume13/desmedt12a/desmedt12a.pdf)
        3. I remove any suggested antonyms that are short phrases (eg Wordnet suggests that "take_away" is an antonym of "add", however as adding a work word change the structure of the sentence.)
        4. select one randomly if there are multiple.
4. I incrementally subsitute one addional antynym until I have run out of words with antonys, saving the sentence at each step. (What I'm currentl;y callign each sentence of different corruption level)
5. I repair the indefinite articles ('an' vs 'a')

I can check the final sentence by sending it through the POS tagger and seeing if I get the same tags.
This is not perfect. Its not bad though.
It got a lot better when I changed to using the Stanford POS tagger, as it was more able to tag and retag correctly and thus was most consistent.

The whole method is very heurustic.
It's not too bad but makes a lot of unnatural sounding sentences.
And will sometimes choose the wrong antonym.

For example stating with the sentence:
 - It's not too bad but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing acquittals.

In [2]:
from SemanticCorruption import *

In [3]:
from nltk.corpus import wordnet as wn
import pattern.en as en

In [89]:
sents = ["It's not too bad but makes a lot of unnatural sounding sentences.",
'A Washington County man may have the countys first human case of West Nile virus, the health department said Friday',
"It was a dusty, dry, hot day, and the flys were buzzing.",
"The article is the most common determiner (DT) in English.",
"We may have a question",
"Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?",
"Both gerunds and infinitives can be used as the subject or the complement of a sentence.",
"Shares of Xoma fell 16 percent in early trade, while shares of Genentech, a much larger company with several products on the market, were up 2 percent.",
"Six months ago, the IMF and Argentina struck a bare-minimum $6.8-billion debt rollover deal that expires in August.",
"He plans to have dinner with troops at Kosovo's U.S. military headquarters, Camp Bondsteel.",
"After that, he plans to have dinner at Camp Bondsteel with U.S. troops stationed there.",
"He added that prosecutors will seek the death penalty.",
"Who is this man?",
"Who is that man?",
"This evil thief stole that car!",
"The motorist is angry, so the pedestrian is understandably scared",
"The hero was brave, he defeated the dragon.",
"The villian was cowardly, he was slain by the dragon.",
"War and Peace opens in the Russian city of St. Petersburg in 1805, as Napoleon's conquest of western Europe is just beginning to stir fears in Russia.",
"PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer."
        ]

In [16]:
def leveled_semantic_corrupt_verb_antonym_sentences(sent):
    return leveled_semantic_corrupt_sentences(sent, get_verb_antos)

def leveled_semantic_corrupt_noun_synonym_sentences(sent):
    return leveled_semantic_corrupt_sentences(sent, get_noun_synonyms)

In [17]:
for sent in sents:

    print(sent)
    print("\n".join(list(leveled_semantic_corrupt_verb_antonym_sentences(sent))))
    print("-"*50)

It's not too bad but makes a lot of unnatural sounding sentences.
It 's not too bad but makes a lot of unnatural devoicing sentences .
--------------------------------------------------
A Washington County man may have the countys first human case of West Nile virus, the health department said Friday

--------------------------------------------------
It was a dusty, dry, hot day, and the flys were buzzing.

--------------------------------------------------
The article is the most common determiner (DT) in English.

--------------------------------------------------
We may have a question

--------------------------------------------------
Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?
Is staying an odd number of verbs , adverbs , adjectives and nouns to their antonyms expected to produce a semantically distant sentence ?
--------------------------------------------------
Both gerunds and infinit

In [80]:
def n_wise(n, seq):
    """ Generalisation of pairwise"""
    from collections import deque
    prevs = deque(maxlen=n)
    for ii,ele in enumerate(seq,1):
        prevs.append(ele)
        if ii>=n:
            yield tuple(prevs)
        

In [90]:
def get_phrases(tagged_sent, max_phrase_length):
    for phrase_len in range(2, max_phrase_length+1):
        for tuple_of_tagged_words in n_wise(phrase_len, tagged_sent):
            words, tags = zip(*tuple_of_tagged_words)
            possible_phrase = "_".join(words)
            if wn.synsets(possible_phrase): #If there are any, then it is a phrase
                phrase = possible_phrase.split("_")
                pos = tags[-1]
                yield(phrase, pos)


In [86]:
def get_tagged_phrases(tagged_sent, max_phrase_length):
    tagged_phrase_sent = list(tagged_sent)
    for phrase_len in range(max_phrase_length,1,-1): #Go from largest to smallest to keep information
        for indexes in n_wise(phrase_len, range(len(tagged_sent))):
            tagged_words = [tagged_phrase_sent[index] for index in indexes]
            if not(any([tagged_word is None for tagged_word in tagged_words])):
                words, tags = zip(*tagged_words)
                possible_phrase = "_".join(words)
                if wn.synsets(possible_phrase): #If there are any, then it is a phrase
                    for index in indexes:
                        tagged_phrase_sent[index] = None #Blank them out with Nones which we will remove later
                    pos = tags[-1] #Use final tag, it will be the one we need for handling plurals
                    tagged_phrase_sent[indexes[0]] = (possible_phrase, pos)
    return [tagged_phrase for tagged_phrase in tagged_phrase_sent if not tagged_phrase is None]
                    


In [87]:
list(

[(u'PCCW', u'NNP'),
 (u"'s", u'POS'),
 (u'chief_operating_officer', u'NN'),
 (u',', u','),
 (u'Mike', u'NNP'),
 (u'Butcher', u'NNP'),
 (u',', u','),
 (u'and', u'CC'),
 (u'the', u'DT'),
 (u'Arena', u'NNP'),
 (u'brothers', u'NNS'),
 (u',', u','),
 (u'the', u'DT'),
 (u'chief_financial_officers', u'NNS'),
 (u',', u','),
 (u'will', u'MD'),
 (u'report', u'VB'),
 (u'directly', u'RB'),
 (u'to', u'TO'),
 (u'the', u'DT'),
 (u'police_officer', u'NN'),
 (u'.', u'.')]

In [94]:
for sent in sents:
    tagged_phrases = get_tagged_phrases(tokenize_and_tag(sent)[1],3)
    phrases = zip(*tagged_phrases)[0]
    print " ".join(phrases)

It 's not too_bad but makes a_lot of unnatural sounding sentences .
A Washington County man may have the countys first human case of West_Nile_virus , the health department said Friday
It was a dusty , dry , hot day , and the flys were buzzing .
The article is the most common determiner ( DT ) in English .
We may have a question
Is changing an odd number of verbs , adverbs , adjectives and nouns to their antonyms expected to produce a semantically distant sentence ?
Both gerunds and infinitives can be used as the subject or the complement of a sentence .
Shares of Xoma fell 16 percent in early trade , while shares of Genentech , a much larger company with several products on the market , were up 2 percent .
Six months ago , the IMF and Argentina struck a bare-minimum $ 6.8-billion debt rollover deal that expires in August .
He plans to have dinner with troops at Kosovo 's U.S. military_headquarters , Camp Bondsteel .
After that , he plans to have dinner at Camp Bondsteel with U.S. troo

In [97]:
list(get_all_synonyms("military_headquarters"))

[u'headquarters', u'HQ', u'military_headquarters']